In [1]:
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import time

Daily meteorological information obtained from https://open-meteo.com/

In [2]:
def generate_link(df):
    lat = df.centroid_lat
    lon = df.centroid_lon
    meteorological_features = [
        'weathercode',
        'temperature_2m_max',
        'temperature_2m_min',
        'temperature_2m_mean',
        'apparent_temperature_max',
        'apparent_temperature_min',
        'apparent_temperature_mean',
        'sunrise',
        'sunset',
        'shortwave_radiation_sum',
        'precipitation_sum',
        'rain_sum',
        'precipitation_hours',
    ]
    meteorological_features_link_text = ','.join(meteorological_features)
    
    link_parts = [
        'https://archive-api.open-meteo.com/v1/archive?',
        f'latitude={lat}&longitude={lon}',
        '&start_date=1980-01-01&end_date=2023-06-15&',
        f'daily={meteorological_features_link_text}&timezone=GMT',
        
    ]
    complete_link = ''.join(link_parts)
    return complete_link

In [3]:
data = gpd.read_file('../data/raw/00mun.shp')

In [4]:
data

,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry
0,01001,01,001,Aguascalientes,"POLYGON ((2489072.503 1115771.584, 2489352.774..."
1,01002,01,002,Asientos,"POLYGON ((2494680.261 1141224.505, 2494749.948..."
2,01003,01,003,Calvillo,"POLYGON ((2429607.454 1120262.254, 2429071.902..."
3,01004,01,004,CosÃ­o,"POLYGON ((2470517.824 1155028.588, 2470552.248..."
4,01005,01,005,JesÃºs MarÃ­a,"POLYGON ((2465526.729 1114740.466, 2465752.546..."
...,...,...,...,...,...
2464,32053,32,053,Villa GonzÃ¡lez Ortega,"POLYGON ((2504476.172 1180670.127, 2504478.046..."
2465,32054,32,054,Villa Hidalgo,"POLYGON ((2531223.368 1166188.348, 2531242.386..."
2466,32055,32,055,Villanueva,"POLYGON ((2420011.778 1183552.394, 2420043.461..."
2467,32056,32,056,Zacatecas,"POLYGON ((2421707.751 1202019.971, 2421780.086..."


#The WGS84 Coordinate Systems adds Greenwich as the starting point (prime meridian) for the longitude (0°) and sets the units in degrees (°). This coordinate system also has a unique reference code, the so-called EPSG code, which is 4326


In [81]:
centroids = data.centroid.to_crs(epsg='4326')
centroids_lon = centroids.apply(lambda x:x.coords[0][0])
centroids_lat = centroids.apply(lambda x:x.coords[0][1])

data['centroid_lat'] = centroids_lat
data['centroid_lon'] = centroids_lon

In [83]:
data_link = data.apply(generate_link, axis=1)

In [86]:
def obtain_link_data(link):
    link_response = requests.get(link)
    my_json = link_response.content.decode('utf8')
    link_data = json.loads(my_json)
    time.sleep(60)
    return link_data

In [87]:
informational_return = data_link.apply(obtain_link_data)

In [88]:
data['data_link'] = data_link

In [89]:
data['meteorological_information'] = informational_return

In [94]:
desired_meteorological_information_cols = ['CVEGEO', 'CVE_ENT', 'CVE_MUN', 'NOMGEO', 'centroid_lat', 'centroid_lon', 'data_link', 'meteorological_information']
data[desired_meteorological_information_cols].to_csv('../data/raw/meteorological_information.csv')